- Проверить наличие в справочниках адресов, брендов, граммовок, регионов (если нет добавить)
- Взять файл
- Заменить адреса, субъекты, бренды, вес на корректные
- Объединить по месяцу, году, региону, бренду, адресу, сумма по весу
- то же по конкурентам
- объединить их вместе
- выгрузить в файл
- убрать адрес Санкт Петербург, Пироговская наб., д.17
- сделать сводный

In [1]:
import pandas as pd

from datetime import datetime

import xlrd

### импорт нужных таблиц

In [249]:
path = 'D:\\Analysis Burn'

In [250]:
burn = pd.read_csv(path + '\\Википедия\\БЁРН исходники\\burn_tobacco.csv')
dsmh = pd.read_csv(path + '\\Википедия\\СВМХ исходники\\dsmh.csv')

In [251]:
addresses = pd.read_excel(path + '\\Википедия\\Справочники\\addresses.xlsx')

In [252]:
regions = pd.read_excel(path + '\\Википедия\\Справочники\\regions.xlsx')

In [253]:
brands = pd.read_excel(path + '\\Википедия\\Справочники\\brands.xlsx')

In [254]:
weight = pd.read_excel(path + '\\Википедия\\Справочники\\weight.xlsx')

In [255]:
burn_products_price = pd.read_excel(r"D:\Analysis Burn\Википедия\Справочники\burn_products_price.xlsx")

In [256]:
dsmh_products_price = pd.read_excel(r"D:\Analysis Burn\Википедия\Справочники\dsmh_products_price.xlsx")

# Получение отсутствующих значений в справочниках

In [257]:
addresses_bad = pd.concat([dsmh['адрес'], burn['Адрес']] )

In [258]:
addresses_bad = addresses_bad.drop_duplicates()
addresses_bad = addresses_bad.dropna()

In [259]:
addresses_bad = pd.DataFrame(addresses_bad)

In [260]:
addresses_bad = addresses_bad.rename(columns={0:'Адрес'})

In [261]:
merged = pd.merge(addresses, addresses_bad, left_on='Исходный адрес', right_on = 'Адрес', how='outer', indicator=True)

In [262]:
missing_values = merged.loc[ merged['_merge'] == 'right_only' ]

In [263]:
if missing_values.empty == False:
    missing_values.to_excel(r'C:\Users\User\Desktop\missing_addresses.xlsx', index=False)
else:
    print("Он пустой")

Он пустой


In [264]:
regions_bad = pd.concat([dsmh['рег'], burn['Субъект РФ']] )

In [265]:
regions_bad = regions_bad.drop_duplicates()
regions_bad = regions_bad.dropna()

In [266]:
regions_bad = pd.DataFrame(regions_bad)

In [267]:
regions_bad =regions_bad.rename(columns={0:'Регион'})

In [268]:
merged2 = pd.merge(regions, regions_bad, left_on='Субъект РФ исходный', right_on = 'Регион', how='outer', indicator=True)

In [269]:
missing_values2 = merged2.loc[merged2['_merge'] == 'right_only', 'Регион']

In [270]:
if missing_values2.empty == False and missing_values2.iloc[0] != 'Неизвестно':
    missing_values2.to_excel(r'C:\Users\User\Desktop\missing_regions.xlsx')
else:
    print("Он пустой")

Он пустой


In [271]:
brands_bad = pd.concat([dsmh['бренд'], burn['Наименование товара']] )

In [272]:
brands_bad = brands_bad.drop_duplicates()
brands_bad = brands_bad.dropna()

In [273]:
brands_bad = pd.DataFrame(brands_bad)

In [274]:
brands_bad = brands_bad.rename(columns={0:'Бренд'})

In [275]:
merged3 = pd.merge(brands, brands_bad, left_on='Бренд (исходный)', right_on = 'Бренд', how='outer', indicator=True)

In [276]:
missing_values3 = merged3.loc[merged3['_merge'] == 'right_only', 'Бренд_y']

In [277]:
if missing_values3.empty == False:
    missing_values3.to_excel(r'C:\Users\User\Desktop\missing_brands.xlsx')
else:
    print("Он пустой")

Он пустой


In [278]:
weight_bad = pd.concat([dsmh['вес'], burn['Наименование товара']] )

In [279]:
weight_bad = weight_bad.drop_duplicates()

In [280]:
weight_bad = pd.DataFrame(weight_bad)

In [281]:
weight_bad = weight_bad.rename(columns={0:'Граммовка'})

In [282]:
merged4 = pd.merge(weight, weight_bad, left_on='Исходная граммовка', right_on = 'Граммовка', how='outer', indicator=True)

In [283]:
missing_values4 = merged4.loc[merged4['_merge'] == 'right_only', 'Граммовка_y']

In [284]:
if missing_values4.empty == False:
    missing_values4.to_excel(r'C:\Users\User\Desktop\missing_weight.xlsx')
else:
    print("Он пустой")

Он пустой


### Проверка наличия цен

In [285]:
dsmh_products_price['Наименование товара'] = dsmh_products_price['продукт']+dsmh_products_price['вес'].astype(str)
dsmh_products_price = dsmh_products_price[['продукт','Наименование товара', 'Цена']]

In [286]:
burn_products_price['продукт'] = burn_products_price['Наименование товара']
burn_products_price = burn_products_price[['продукт', 'Наименование товара','Цена']]

In [287]:
prices = pd.concat([dsmh_products_price, burn_products_price] )

In [288]:
prices = prices.drop_duplicates()
prices = prices.dropna()

In [289]:
prices = pd.DataFrame(prices)

In [290]:
burn_product = pd.DataFrame()
burn_product['Наименование товара'] = burn['Наименование товара']
burn_product['без веса']  = pd.NaT
burn_product['Вес (гр)'] = pd.NaT


In [291]:
dsmh_product = pd.DataFrame()
dsmh_product['Наименование товара'] = dsmh['продукт']+dsmh['вес'].astype(str)
dsmh_product['без веса'] = dsmh['продукт']
dsmh_product['Вес (гр)']  = dsmh['вес'].astype(str)

In [292]:
products_for_check = pd.concat([burn_product, dsmh_product])
products_for_check = products_for_check.drop_duplicates()
products_for_check = products_for_check.dropna()

In [293]:
merged = pd.merge(prices, products_for_check, on='Наименование товара', how='outer', indicator=True)

In [294]:
missing_values5 = merged.loc[ merged['_merge'] == 'right_only' ]

In [295]:
if missing_values5.empty == False:
    missing_values5.to_excel(r'C:\Users\User\Desktop\missing_prices.xlsx', index=False)
else:
    # здесь если не пусто, то удалять в конце цифры
    print("Он пустой")
    

Он пустой


# РАБОТА С ФАЙЛОМ BURN

### адреса из справочника

In [296]:
need_addresses = addresses[['Исходный адрес', 'Адрес']]

In [297]:
burn_good_address = burn.merge(need_addresses, how="left", left_on='Адрес', right_on='Исходный адрес')

In [298]:
burn_good_address = burn_good_address.drop(columns=['Исходный адрес', 'Адрес_x'])

In [299]:
burn_good_address = burn_good_address.rename(columns={"Адрес_y": "Адрес"})

### регионы(субъекты) из справочника

In [300]:
burn_good_address = burn_good_address.merge(
    regions, how="left", left_on="Субъект РФ", right_on="Субъект РФ исходный", suffixes=('_left', '_right')
)

In [301]:
burn_good_address = burn_good_address.drop(columns=['Субъект РФ исходный', 'Субъект РФ_left'])

In [302]:
burn_good_address = burn_good_address.rename(columns={"Субъект РФ_right": "Субъект РФ"})

### изменить тип на дату и вычленить дату

In [303]:
burn_good_address['Дата операции'] = burn_good_address['Дата операции'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d')
)

In [304]:
burn_good_address['Месяц'] = burn_good_address['Дата операции'].dt.month

In [305]:
burn_good_address['Год'] = burn_good_address['Дата операции'].dt.year

### заменить бренды из справочника

In [306]:
burn_good_address = burn_good_address.merge(
    brands, how="left", left_on="Наименование товара", right_on="Бренд (исходный)", suffixes=('_left', '_right')
)

In [307]:
burn_good_address = burn_good_address.drop(columns=['Бренд (исходный)'])

### заменить граммовку

In [308]:
burn_good_address = burn_good_address.merge(
    weight, how="left", left_on="Наименование товара", right_on="Исходная граммовка", suffixes=('_left', '_right')
)

In [309]:
burn_good_address = burn_good_address.drop(columns=['Исходная граммовка'])

### Добавление колонки вес

In [310]:
burn_good_address['Граммовка'] = burn_good_address['Граммовка'].astype(int)

In [311]:
burn_good_address['Вес'] = (burn_good_address['Граммовка']/1000)* burn_good_address['Объем товара (пачек)']

In [312]:
burn_good_address['Адрес'] = burn_good_address['Адрес'].fillna('не указан')

### цены из справочника

In [313]:
burn_good_address = burn_good_address.merge(prices, how="left", on='Наименование товара')

In [314]:
# должен быть ноль
burn_good_address['Цена'].isna().sum()

4519

In [315]:
burn_good_address['Цена'] = burn_good_address['Цена'] * burn_good_address['Объем товара (пачек)']

### Группировка

In [316]:
burn_good = burn_good_address.groupby(['Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес']).agg({'Вес':'sum','Цена':'sum'}).reset_index()

In [317]:
burn_good.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252075 entries, 0 to 252074
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Год         252075 non-null  int64  
 1   Месяц       252075 non-null  int64  
 2   Субъект РФ  252075 non-null  object 
 3   Бренд       252075 non-null  object 
 4   Адрес       252075 non-null  object 
 5   Вес         252075 non-null  float64
 6   Цена        252075 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 13.5+ MB


# РАБОТА С ФАЙЛОМ MHDS

In [318]:
dsmh_copy = dsmh.copy(deep = True)

### Выделить месяц и год

In [319]:
dsmh_copy['Дата операции'] = dsmh_copy['мес'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d')
)

In [320]:
dsmh_copy['Месяц'] = dsmh_copy['Дата операции'].dt.month

In [321]:
dsmh_copy['Год'] = dsmh_copy['Дата операции'].dt.year

### регион из справочника

In [322]:
dsmh_copy = dsmh_copy.merge(
    regions, how="left", left_on="рег", right_on="Субъект РФ исходный", suffixes=('_left', '_right')
)

In [323]:
dsmh_copy = dsmh_copy.drop(columns=['Субъект РФ исходный', 'рег'])

### адрес из справочника

In [324]:
need_addresses = addresses[['Исходный адрес', 'Адрес']]

In [325]:
dsmh_good_address = dsmh_copy.merge(need_addresses, how="left", left_on='адрес', right_on='Исходный адрес')

In [326]:
dsmh_good_address = dsmh_good_address.drop(columns=['Исходный адрес', 'адрес'])

### цены из справочника

In [327]:
dsmh_good_address['Наименование товара'] = dsmh_good_address['продукт'] + dsmh_good_address['вес'].astype(str)

In [328]:
dsmh_good_address = dsmh_good_address.merge(prices, how="left", on='Наименование товара')

In [329]:
dsmh_good_address['Цена'].isna().sum()

0

In [330]:
dsmh_good_address['Цена'] = dsmh_good_address['Цена'] * dsmh_good_address['колво']

### вес из кол-во и граммовки (у них вес называется)

In [331]:
dsmh_good_address['Новый вес'] = (dsmh_good_address['вес']/1000) * dsmh_good_address['колво']

In [332]:
dsmh_good_address = dsmh_good_address.drop(columns=['вес', 'колво'])

In [333]:
dsmh_good_address = dsmh_good_address.rename(columns = {'Новый вес': 'Вес'})

### бренд из справочника (менять не надо)

In [334]:
dsmh_good_address = dsmh_good_address.rename(columns={'бренд':'Бренд'})

### Группировка

In [335]:
dsmh_good_address = dsmh_good_address.groupby(['Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес']).agg({'Вес':'sum','Цена':'sum'}).reset_index()

In [336]:
dsmh_good_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414237 entries, 0 to 414236
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Год         414237 non-null  int64  
 1   Месяц       414237 non-null  int64  
 2   Субъект РФ  414237 non-null  object 
 3   Бренд       414237 non-null  object 
 4   Адрес       414237 non-null  object 
 5   Вес         414237 non-null  float64
 6   Цена        414237 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 22.1+ MB


In [337]:
(dsmh_good_address.columns == burn_good.columns).all()

True

In [338]:
BBDSMH = pd.concat([dsmh_good_address, burn_good], ignore_index=True)

In [339]:
# удалить ненужный адрес
BBDSMH = BBDSMH.loc[ BBDSMH['Адрес'] != 'г Санкт-Петербург, наб Пироговская, д 17 к 7' ]
BBDSMH = BBDSMH.loc[ BBDSMH['Адрес'] != 'г Санкт-Петербург, наб Пироговская, д 17 к 7 литера а' ]


In [342]:
BBDSMH = BBDSMH.loc[ (BBDSMH['Бренд'] == 'DARKSIDE') | (BBDSMH['Бренд'] == 'MUST HAVE') | (BBDSMH['Бренд'] == 'Blackburn') | (BBDSMH['Бренд'] == 'SPECTRUM TOBACCO') ]

In [345]:
BBDSMH.loc[BBDSMH['Бренд'] == 'DARKSIDE', 'Бренд'] = "darkside"
BBDSMH.loc[BBDSMH['Бренд'] == 'MUST HAVE', 'Бренд'] = "must have"
BBDSMH.loc[BBDSMH['Бренд'] == 'SPECTRUM TOBACCO', 'Бренд'] = "spectrum tobacco"
BBDSMH.loc[BBDSMH['Бренд'] == 'Blackburn', 'Бренд'] = "BLACKBURN"

In [346]:
BBDSMH = BBDSMH.merge(
    addresses[['Адрес', 'кб']],
    how='left', 
    on='Адрес'
)

In [347]:
BBDSMH = BBDSMH.loc[(BBDSMH['кб'] == 'нет') | (BBDSMH['кб'].isna()) ]

In [348]:
BBDSMH = BBDSMH[[
    'Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес', 'Вес', 'Цена'
]]

In [349]:
BBDSMH.to_excel(r'D:\Analysis Burn\Википедия\Wikipedia.xlsx', index=False)